In [16]:
import pandas as pd
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord


In [21]:
hallmart_hits = pd.read_csv('./2.Hallmarks_vs_NegData/neg_data_recaught_hits.csv', sep=',', header=0)
print(hallmart_hits.shape)
print(hallmart_hits.dtypes)
hallmart_hits.head()

(16, 5)
sseqid     object
sstart      int64
send        int64
sstrand    object
sseq       object
dtype: object


,sseqid,sstart,send,sstrand,sseq
0,LinJ.16,64229,64998,plus,CCCGTGCTGCGTCGCGCCCATCTAGCTGCCACGGCGGCATGGCTCC...
1,LinJ.16,79840,80609,plus,CCCGTGCTGCGTCGCGCCCATCTAGCTGCCACGGCGGCATGGCTCC...
2,LinJ.16,79841,80335,plus,CCGTGCTGCGTCGCGCCCATCTAGCTGCCACGGCGGCATGGCTCCA...
3,LinJ.17,594954,595099,plus,GCGTGGCATCTGAGGGTCCCGCACCCAGCTCTCCGTGTGGAAGGCG...
4,LinJ.17,638017,638178,plus,GGGCACCCCTCTCCGCGCGTGGCGTCTGAGGGTCCCGCACCCAGCT...


In [22]:
neg_data = pd.read_csv('./2.Hallmarks_vs_NegData/negative_data_after_recaught.csv', sep=',', header=0)
print(neg_data.shape)
print(neg_data.dtypes)
neg_data.head()

(898, 5)
sseqid     object
sstart      int64
send        int64
sstrand    object
sseq       object
dtype: object


,sseqid,sstart,send,sstrand,sseq
0,LinJ.01,36104,36243,plus,GACAGACCGACACACGCAGCCGTGTGATGCCGCCGCCGAGGGCAGT...
1,LinJ.01,146413,146531,plus,CGAATTGTGTTCTGCGCATGCCTCTTCTCTGCCGTGCAGCATGCGG...
2,LinJ.01,271364,271651,plus,GAACGCCGCCCTCAATCGCGCGCTGAACTTCACGCGGCGGTCGACC...
3,LinJ.02,17014,17130,plus,GGAGGGGACGCGCGGAGCTGCGATGCGAAAGTGAGAGCAACACAGA...
4,LinJ.02,74330,74460,plus,TGTCACAAGCGACTCGAAGAGGACGAAAAGACACACGGCCGCACAC...


In [23]:
old_neg_data = pd.read_csv('./new_data/positive_database.csv', sep=',', header=0)
print(old_neg_data.shape)
print(old_neg_data.dtypes)
old_neg_data.head()

(2116, 5)
sseqid     object
sstart      int64
send        int64
sstrand    object
sseq       object
dtype: object


,sseqid,sstart,send,sstrand,sseq
0,LinJ.01,2,174,plus,CACCAGTACACCAGTACACCAGTACACCAGTACACCAGTACACCAG...
1,LinJ.01,24094,24759,plus,GGGGAGGCGGGGGAGGCGGGGGGCACGCACCTCCATGCGTGGCATC...
2,LinJ.01,35372,35957,plus,CTCCCATCGCCTGGTGCGAAGCAGCGCAAGACACACGCGCGCTGCA...
3,LinJ.01,39791,40596,plus,TTCTACCGCGAGCAAGGCAGCACACAGACGCACGCACAGCCACAGC...
4,LinJ.01,54984,55548,plus,CTCTCATCGCCTGGTGCGAAGCAGCGCAAGACACACGCGCGCTGCA...


In [25]:
# Adding re-caught data to the new positive database
new_neg_data = pd.concat([old_neg_data, hallmart_hits], ignore_index=True, axis=0)
new_neg_data.sort_values(by=['sseqid', 'sstart'], inplace=True)
print(new_neg_data.shape)
print(new_neg_data.dtypes)
new_neg_data.head()

(2132, 5)
sseqid     object
sstart      int64
send        int64
sstrand    object
sseq       object
dtype: object


,sseqid,sstart,send,sstrand,sseq
0,LinJ.01,2,174,plus,CACCAGTACACCAGTACACCAGTACACCAGTACACCAGTACACCAG...
1,LinJ.01,24094,24759,plus,GGGGAGGCGGGGGAGGCGGGGGGCACGCACCTCCATGCGTGGCATC...
2,LinJ.01,35372,35957,plus,CTCCCATCGCCTGGTGCGAAGCAGCGCAAGACACACGCGCGCTGCA...
3,LinJ.01,39791,40596,plus,TTCTACCGCGAGCAAGGCAGCACACAGACGCACGCACAGCCACAGC...
4,LinJ.01,54984,55548,plus,CTCTCATCGCCTGGTGCGAAGCAGCGCAAGACACACGCGCGCTGCA...


In [26]:
new_neg_data.to_csv('./new_data_v2/negative_database.csv', sep=',', index=False, header=True)
neg_data.to_csv('./new_data_v2/negative_data.csv', sep=',', index=False, header=True)